In [12]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import copy
import multiprocessing

In [13]:
#energy calculation of a column (row.T)
def energy(image):
    if len(image.shape) == 3:
        energy_map = np.zeros(image.shape[:2], dtype=np.float64)

        for channel in range(3):
            sobel_x = cv.Sobel(image[:, :, channel], cv.CV_64F, 1, 0, ksize=3)
            sobel_y = cv.Sobel(image[:, :, channel], cv.CV_64F, 0, 1, ksize=3)
            energy_map += np.sqrt(sobel_x**2 + sobel_y**2)

    else:
        sobel_x = cv.Sobel(image, cv.CV_64F, 1, 0, ksize=3)
        sobel_y = cv.Sobel(image, cv.CV_64F, 0, 1, ksize=3)
        energy_map = np.sqrt(sobel_x**2 + sobel_y**2)

    return energy_map

In [14]:
#remove a seam (row.T)
import numpy as np

def remove_seam(image, indexes):
    rows, cols, channels = image.shape
    indexes = np.array(indexes)  # Ensure it's a NumPy array

    if len(indexes) != rows:
        raise ValueError(f"Mismatch: Expected {rows} seam indexes but got {len(indexes)}")

    # Create a mask to keep only necessary pixels
    mask = np.ones((rows, cols), dtype=bool)

    for i in range(rows):
        j = indexes[i]
        if not (0 <= j < cols):
            raise ValueError(f"Invalid index {j} for row {i}, must be between 0 and {cols - 1}")
        mask[i, j] = False  # Mark pixel for removal

    # Apply mask to remove seam
    new_image = image[mask].reshape((rows, cols - 1, channels))

    return new_image

In [15]:
# sequential Dynamic programming
import numpy as np

def min_index(p_l, p, p_r):
    if p_l <= p and p_l <= p_r:
        return -1
    elif p <= p_l and p <= p_r:
        return 0
    else:
        return 1

def sequential_seam_find(image):
    energy_matrix = energy(image)
    rows, cols = energy_matrix.shape

    val = [energy_matrix[rows - 2][j] for j in range(1, cols - 1)]
    indexes = [[j] for j in range(1, cols - 1)]

    for i in range(rows - 3, 0, -1):
        new_val = [0] * (cols - 2)
        new_indexes = [[] for _ in range(cols - 2)]

        for j in range(1, cols - 1):
            p_l = energy_matrix[i][j - 1] if j - 1 >= 1 else float('inf')
            p = energy_matrix[i][j]
            p_r = energy_matrix[i][j + 1] if j + 1 < cols - 1 else float('inf')

            index_offset = min_index(p_l, p, p_r)
            new_val[j - 1] = val[(j - 1) + index_offset] + energy_matrix[i][j]
            new_indexes[j - 1] = indexes[(j - 1) + index_offset] + [j]

        val, indexes = new_val, new_indexes

    return dict(zip(val, indexes))

In [16]:
import torch

def min_index(p_l, p, p_r):
    offsets = torch.tensor([-1, 0, 1], device=p_l.device)
    values = torch.stack([p_l, p, p_r], dim=0)
    min_indices = torch.argmin(values, dim=0)
    return offsets[min_indices]

def parallel_seam_find(image):
    energy_matrix = torch.tensor(energy(image), device='cuda', dtype=torch.float32)
    rows, cols = energy_matrix.shape

    val = energy_matrix[rows - 1, :].clone()
    indexes = [[j] for j in range(cols)]

    for i in range(rows - 2, -1, -1):
        p_l = energy_matrix[i, :-2]
        p = energy_matrix[i, 1:-1]
        p_r = energy_matrix[i, 2:]

        index_offset = min_index(p_l, p, p_r)
        index_offset_list = index_offset.cpu().tolist()

        new_val = val[1:-1] + p
        new_indexes = [indexes[j + index_offset_list[j]] + [j + 1] for j in range(len(new_val))]

        val, indexes = new_val, new_indexes

    min_seam_idx = torch.argmin(val).item()
    seam_path = indexes[min_seam_idx]

    if len(seam_path) != rows:
        print(f"Warning: Seam path length mismatch ({len(seam_path)} vs {rows}). Adjusting...")
        while len(seam_path) < rows:
            seam_path.append(seam_path[-1])  # Extend to maintain length
        seam_path = seam_path[:rows]  # Trim excess if needed

    return seam_path


In [17]:
# remove seam with min score
def remove_min_seam(image):

    seams = parallel_seam_find(image)
    print(seams)
    min_energy = min(seams.keys())
    indexes = seams[min_energy]

    image = remove_seam(image, indexes)

    return image, indexes


In [18]:
# deal with columns
def uni_resize(image, seam_y):
  indexes = []

  for i in range(seam_y):
    image, index = remove_min_seam(image)
  indexes.append(index)

  return image, indexes

In [19]:
def transpose_image(image):
    return np.transpose(image, (1, 0, 2))

In [20]:
def resize(image, percentage):
    rows, cols, _ = image.shape
    print(f"Image dimensions: {rows} x {cols}")
    print(f"Number of CPU cores: {multiprocessing.cpu_count()}")

    num_seam_x = int(rows * percentage)
    num_seam_y = int(cols * percentage)

    image, row_red_indexes = uni_resize(transpose_image(image), num_seam_x)
    image = transpose_image(image)

    image, column_red_indexes = uni_resize(image, num_seam_y)

    return image, row_red_indexes, column_red_indexes

In [21]:
def make_red(image, red_indexes):
    image = copy.deepcopy(image)

    for seam in red_indexes:
        shift = 0
        for row, col in enumerate(seam):
            adjusted_col = col + shift

            adjusted_col = max(0, min(adjusted_col, image.shape[1] - 1))

            image[row, adjusted_col] = [255, 0, 0]

            if row > 0:
                if seam[row] > seam[row - 1]:
                    shift += 1
                elif seam[row] < seam[row - 1]:
                    shift -= 1

    return image

In [ ]:
image = cv.imread('/content/OIP.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

energy_map = energy(image)

resized_image, row_red_indexes, column_red_indexes = resize(image, 0.2)

highlighted_image = make_red(image, row_red_indexes)

highlighted_image = make_red(highlighted_image.transpose(1, 0, 2), column_red_indexes).transpose(1, 0, 2)

fig, axes = plt.subplots(1, 4, figsize=(15, 5))
titles = ["Original", "Energy Map", "Resized", "Deleted Seams"]
images = [image, energy_map, resized_image, highlighted_image]

for ax, img, title in zip(axes, images, titles):
    if img.ndim == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()